In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import timm
import numpy as np
import cv2
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score, confusion_matrix
import os
from tqdm import tqdm
import pandas as pd

torch.cuda.empty_cache()

def create_dataset_df(base_path):
    """Create a DataFrame containing image paths and labels"""
    data = []
    
    # First Set (100x)
    first_set = os.path.join(base_path, 'First Set')
    normal_100x = os.path.join(first_set, '100x Normal Oral Cavity Histopathological Images')
    oscc_100x = os.path.join(first_set, '100x OSCC Histopathological Images')
    
    # Add 100x normal images
    for img_name in os.listdir(normal_100x):
        if img_name.endswith(('.jpg')):
            data.append({
                'path': os.path.join(normal_100x, img_name),
                'label': 0,  # normal
                'magnification': '100x'
            })
    
    # Add 100x OSCC images
    for img_name in os.listdir(oscc_100x):
        if img_name.endswith(('.jpg')):
            data.append({
                'path': os.path.join(oscc_100x, img_name),
                'label': 1,  # OSCC
                'magnification': '100x'
            })
    
    # Second Set (400x)
    second_set = os.path.join(base_path, 'Second Set')
    normal_400x = os.path.join(second_set, '400x Normal Oral Cavity Histopathological Images')
    oscc_400x = os.path.join(second_set, '400x OSCC Histopathological Images')
    
    # Add 400x normal images
    for img_name in os.listdir(normal_400x):
        if img_name.endswith(('.jpg')):
            data.append({
                'path': os.path.join(normal_400x, img_name),
                'label': 0,  # normal
                'magnification': '400x'
            })
    
    # Add 400x OSCC images
    for img_name in os.listdir(oscc_400x):
        if img_name.endswith(('.jpg')):
            data.append({
                'path': os.path.join(oscc_400x, img_name),
                'label': 1,  # OSCC
                'magnification': '400x'
            })
    
    df = pd.DataFrame(data)
    print(f"Total images: {len(df)}")
    print(f"Class distribution:\n{df['label'].value_counts()}")
    print(f"Magnification distribution:\n{df['magnification'].value_counts()}")
    return df

class OralCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = cv2.imread(row['path'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, row['label']

class ConvNextModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.convnext = timm.create_model('convnext_base', pretrained=True, num_classes=num_classes)
        
        # Modify head for binary classification
        self.convnext.head.fc = nn.Sequential(
            nn.LayerNorm(1024),
            nn.Dropout(0.1),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.convnext(x)

def get_transforms(is_train=True):
    """Get transforms for training/validation"""
    size = 224  # ConvNext optimal size
    
    if is_train:
        return A.Compose([
            A.RandomResizedCrop(size, size, scale=(0.8, 1.0)),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5, shift_limit=0.0625, scale_limit=0.1, rotate_limit=45),
            A.OneOf([
                A.GaussNoise(var_limit=(10.0, 50.0)),
                A.GaussianBlur(blur_limit=(3, 7)),
            ], p=0.5),
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            A.Normalize(),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(size, size),
            A.Normalize(),
            ToTensorV2()
        ])

def train_epoch(model, train_loader, optimizer, criterion, device, epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}')
    
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return total_loss / len(train_loader), correct / total

def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            val_preds.extend(probs)
            val_labels.extend(labels.cpu().numpy())
    
    val_preds = np.array(val_preds)
    val_labels = np.array(val_labels)
    
    auc_roc = roc_auc_score(val_labels, val_preds)
    pred_labels = (val_preds > 0.5).astype(int)
    cm = confusion_matrix(val_labels, pred_labels)
    
    sensitivity = cm[1,1] / (cm[1,1] + cm[1,0])
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    
    return {
        'loss': val_loss / len(val_loader),
        'auc_roc': auc_roc,
        'sensitivity': sensitivity,
        'specificity': specificity
    }

def main():
    # Set random seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Parameters
    BATCH_SIZE = 16  # Increased for GPU
    NUM_EPOCHS = 30
    BASE_PATH = '/kaggle/input/histopathological-imaging-oral-cancer-analysis'  # Kaggle dataset path
    
    # Create dataset DataFrame
    df = create_dataset_df(BASE_PATH)
    
    # Split data into train/validation
    from sklearn.model_selection import train_test_split
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
    
    # Create datasets
    train_dataset = OralCancerDataset(train_df, transform=get_transforms(is_train=True))
    val_dataset = OralCancerDataset(val_df, transform=get_transforms(is_train=False))
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                            num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                          num_workers=2, pin_memory=True)
    
    # Initialize model
    device = torch.device('cuda')
    model = ConvNextModel().to(device)
    
    # Calculate class weights for balanced loss
    class_counts = df['label'].value_counts()
    total_samples = len(df)
    class_weights = torch.FloatTensor([total_samples/(2*class_counts[i]) for i in range(2)]).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
    
    # Training loop
    best_auc = 0
    for epoch in range(NUM_EPOCHS):
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device, epoch)
        
        # Validate
        metrics = validate(model, val_loader, criterion, device)
        
        print(f'\nEpoch {epoch + 1}/{NUM_EPOCHS}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {metrics["loss"]:.4f}')
        print(f'Val AUC-ROC: {metrics["auc_roc"]:.4f}')
        print(f'Sensitivity: {metrics["sensitivity"]:.4f}')
        print(f'Specificity: {metrics["specificity"]:.4f}')
        
        # Update scheduler
        scheduler.step()
        
        # Save best model
        if metrics['auc_roc'] > best_auc:
            best_auc = metrics['auc_roc']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_auc': best_auc,
            }, 'best_model.keras')

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Total images: 1224
Class distribution:
label
1    934
0    290
Name: count, dtype: int64
Magnification distribution:
magnification
400x    696
100x    528
Name: count, dtype: int64


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 62/62 [00:52<00:00,  1.17it/s, loss=0.0736, acc=61.39%]



Epoch 1/30:
Train Loss: 0.6833, Train Acc: 0.6139
Val Loss: 0.4435
Val AUC-ROC: 0.8681
Sensitivity: 0.7380
Specificity: 0.8793


...


Epoch 28/30:
Train Loss: 0.0189, Train Acc: 0.9908
Val Loss: 0.6493
Val AUC-ROC: 0.9525
Sensitivity: 0.9412
Specificity: 0.7759


Epoch 29: 100%|██████████| 62/62 [00:51<00:00,  1.21it/s, loss=0.5009, acc=98.88%]



Epoch 29/30:
Train Loss: 0.0305, Train Acc: 0.9888
Val Loss: 0.6477
Val AUC-ROC: 0.9524
Sensitivity: 0.9412
Specificity: 0.7759


Epoch 30: 100%|██████████| 62/62 [00:51<00:00,  1.21it/s, loss=0.0022, acc=98.88%]



Epoch 30/30:
Train Loss: 0.0272, Train Acc: 0.9888
Val Loss: 0.6507
Val AUC-ROC: 0.9524
Sensitivity: 0.9412
Specificity: 0.7586
